In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from functools import reduce

In [65]:
df = pd.read_csv("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,82,114,147,177,212,272,322,411,599,599.0
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,773,839,825,878,889,924,963,1007,1086,1086.0
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,212,226,243,266,313,345,385,432,455,455.0
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,2,2.0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,238,428,566,673,790,900,1030,1183,1306,1306.0


In [66]:
# Ultimate goal: Get data in 3 column format: Date, Country, TotalConfirmedCasesThusFar

# Step 1: Convert date columns to rows
df = df.drop(df.columns[[0, 2, 3]], axis=1)

# Step 2: Group by Country 
dfAggs = df.groupby('Country/Region')

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,Thailand,2,3,5,7,8,8,14,14,14,...,82,114,147,177,212,272,322,411,599,599.0
1,Japan,2,1,2,2,4,4,7,7,11,...,773,839,825,878,889,924,963,1007,1086,1086.0
2,Singapore,0,1,3,3,4,5,7,7,10,...,212,226,243,266,313,345,385,432,455,455.0
3,Nepal,0,0,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2,2.0
4,Malaysia,0,0,0,3,4,4,4,7,8,...,238,428,566,673,790,900,1030,1183,1306,1306.0


In [ ]:
# Step 2: Convert dates to invididual rows using melt()
key_columns = df.columns.to_list()[:1]
date_columns = df.columns.to_list()[1:]

df_clean = pd.melt(
    df
    , id_vars=key_columns
    , value_vars=date_columns
    , var_name='Date'
    , value_name='Confirmed'
)

df_clean.head()

In [73]:
dfAggs.head()

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,Thailand,2,3,5,7,8,8,14,14,14,...,82,114,147,177,212,272,322,411,599,599.0
1,Japan,2,1,2,2,4,4,7,7,11,...,773,839,825,878,889,924,963,1007,1086,1086.0
2,Singapore,0,1,3,3,4,5,7,7,10,...,212,226,243,266,313,345,385,432,455,455.0
3,Nepal,0,0,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2,2.0
4,Malaysia,0,0,0,3,4,4,4,7,8,...,238,428,566,673,790,900,1030,1183,1306,1306.0
5,Canada,0,0,0,0,0,0,1,1,1,...,64,73,103,103,186,231,271,424,424,424.0
6,Australia,0,0,0,0,3,4,4,4,4,...,112,134,171,210,267,307,353,436,533,533.0
7,Australia,0,0,0,0,1,1,1,1,2,...,49,57,71,94,121,121,121,229,296,296.0
8,Australia,0,0,0,0,0,0,0,1,3,...,46,61,68,78,94,144,184,221,221,221.0
9,Cambodia,0,0,0,0,0,1,1,1,1,...,7,7,7,33,35,37,51,53,84,84.0


In [68]:
# Convert "columns" into "rows" using melt()
key_columns = df.columns.to_list()[:4]
date_columns = df.columns.to_list()[4:]

df_clean = pd.melt(
    df
    , id_vars=key_columns
    , value_vars=date_columns
    , var_name='Date'
    , value_name='Confirmed'
)

df_clean.head()

AttributeError: 'Index' object has no attribute 'to_list'

In [ ]:

# Step 4: Make cumulative by day
# Step 5: Remove all rows until a country has at least 100 rows

In [ ]:
# Convert "columns" into "rows" using melt()
key_columns = df.columns.to_list()[:4]
date_columns = df.columns.to_list()[4:]

df_clean = pd.melt(
    df
    , id_vars=key_columns
    , value_vars=date_columns
    , var_name='Date'
    , value_name='Confirmed'
)

df_clean.head()

In [74]:
print(pd.__version__)

0.23.0
